In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import matplotlib.pyplot as plt
import random
from numpy import save

2021-12-13 16:19:27.131147: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-13 16:19:27.131187: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df=pd.read_csv('full_df.csv')

In [3]:
df3= df[(df['labels']== "['H']")&(df.H==1)]

In [4]:
df2= df[(df['labels']!= "['H']")& (df.H==0)]

In [5]:
len(df2)

6189

In [6]:
len(df3)

128

In [7]:
NoHyper=df2.filename

In [8]:
Hyper=df3.filename

In [9]:
def create_dataset(image_category,label):
    dataset_dir = "Datos/preprocessed_2"
    image_size=250
    labels = []
    dataset = []
    for img in image_category:
        image_path = os.path.join(dataset_dir,img)
        try:
            img = cv2.imread(image_path,0)
            img = cv2.resize(img, (250,250))
            img = img.astype(np.float32)/255
            img = img[..., np.newaxis]
            
        except:
            continue
        
        dataset.append([np.array(img),np.array(label)])
    random.shuffle(dataset)
    return dataset

In [10]:
def augment(category,label,mult):
    x=[]
    for img in category:
        try:
            img = cv2.imread(f"Datos/preprocessed_2/{img}",0)
            img = cv2.resize(img, (250,250))
            img = img.astype(np.float32)/255
            img = img[..., np.newaxis]
        except:
            continue
        train_datagen = ImageDataGenerator(
                    rotation_range=30,
                    zoom_range=0.1,
                    vertical_flip=True,
                    brightness_range=(0.2, 1.5),
                    height_shift_range=0.1,
                    )
        img = img.reshape((1,) + img.shape)
        
        it = train_datagen.flow(x=img, batch_size=1)
        for i in range(mult):
            batch = it.next()
            aug_img = batch[0]
            aug_img=aug_img.reshape(-1)
            x.append([np.array(img),np.array(label)])
            
       
    for i in range(len(x)):
        x[i][0]=x[i][0].reshape(x[i][0].shape[1:])
    return x

In [15]:
HypertensionAug=augment(Hyper,1, 47)

In [16]:
len(HypertensionAug)

6016

In [13]:
Nohypertension=create_dataset(NoHyper, 0)

In [14]:
Hypertension=create_dataset(Hyper,1)

In [17]:
TodoHyper=HypertensionAug+Nohypertension+Hypertension

In [21]:
x = np.array([i[0]for i in TodoHyper])
y = np.array([i[1] for i in TodoHyper])

In [22]:
x.shape

(12333, 250, 250, 1)

In [23]:
save('x',x)

In [24]:
save('y',y)